# Tabular Playground Series - May 2022

## Keras - 1 Attempt

## Attempt 1

In [2]:
# Reproducibility
import os
import random
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC

# Set random seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_df = pd.read_csv('tabular-playground-series-may-2022/train.csv.zip')
test_df = pd.read_csv('tabular-playground-series-may-2022/test.csv.zip')
sample_sub = pd.read_csv('tabular-playground-series-may-2022/sample_submission.csv.zip')

# Infer columns
id_col = sample_sub.columns[0]
target_col = sample_sub.columns[1]

# Prepare dataframes
df = train_df.copy()

# Encode target
e = LabelEncoder()
y = e.fit_transform(df[target_col].astype(str))

# Separate features and test set
X = df.drop(columns=[id_col, target_col], errors='ignore')
test_ids = test_df[id_col]
X_test = test_df.drop(columns=[id_col, target_col], errors='ignore')

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Feature engineering: drop columns with all missing values
X_train = X_train.dropna(axis=1, how='all')
X_val = X_val[X_train.columns]
X_test = X_test[X_train.columns]

# Identify numeric and categorical features
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = [
    col for col in X_train.select_dtypes(include=['object', 'category']).columns
    if X_train[col].nunique() <= 50
]

# Build preprocessing pipelines
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# Determine model architecture based on feature size
n_features = X_train_proc.shape[1]
hidden_units = [min(n_features * 2, 128), min(n_features, 64)]
dropout_rate = 0.3

# Build the model
model = Sequential([
    Dense(hidden_units[0], activation='relu', input_shape=(n_features,)),
    Dropout(dropout_rate),
    Dense(hidden_units[1], activation='relu'),
    Dropout(dropout_rate),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train the model
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_data=(X_val_proc, y_val),
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Log the final epoch results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Make predictions on the test set
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final_preds = (probs > 0.5).astype(int)

submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final_preds})
submission.to_csv('submission_result.csv', index=False)

2025-07-10 13:26:31.656372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752153991.673401  966912 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752153991.678555  966912 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752153991.692904  966912 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752153991.692919  966912 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752153991.692921  966912 computation_placer.cc:177] computation placer alr

Epoch 1/100


5625/5625 - 9s - 2ms/step - accuracy: 0.7240 - auc: 0.7982 - loss: 0.5467 - val_accuracy: 0.7826 - val_auc: 0.8616 - val_loss: 0.4728
Epoch 2/100


5625/5625 - 9s - 2ms/step - accuracy: 0.7802 - auc: 0.8542 - loss: 0.4778 - val_accuracy: 0.8218 - val_auc: 0.8967 - val_loss: 0.4164
Epoch 3/100


5625/5625 - 8s - 1ms/step - accuracy: 0.8007 - auc: 0.8735 - loss: 0.4492 - val_accuracy: 0.8376 - val_auc: 0.9091 - val_loss: 0.3944
Epoch 4/100


...

5625/5625 - 8s - 1ms/step - accuracy: 0.8268 - auc: 0.8965 - loss: 0.4114 - val_accuracy: 0.8558 - val_auc: 0.9228 - val_loss: 0.3687
Epoch 23/100


5625/5625 - 9s - 2ms/step - accuracy: 0.8272 - auc: 0.8967 - loss: 0.4109 - val_accuracy: 0.8550 - val_auc: 0.9225 - val_loss: 0.3686
Epoch 24/100


5625/5625 - 8s - 2ms/step - accuracy: 0.8274 - auc: 0.8971 - loss: 0.4102 - val_accuracy: 0.8556 - val_auc: 0.9229 - val_loss: 0.3678
Epoch 25/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8279 - auc: 0.8972 - loss: 0.4101 - val_accuracy: 0.8558 - val_auc: 0.9230 - val_loss: 0.3679
Epoch 26/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8272 - auc: 0.8971 - loss: 0.4106 - val_accuracy: 0.8556 - val_auc: 0.9232 - val_loss: 0.3690
Epoch 27/100


5625/5625 - 8s - 1ms/step - accuracy: 0.8276 - auc: 0.8975 - loss: 0.4095 - val_accuracy: 0.8560 - val_auc: 0.9235 - val_loss: 0.3663
Epoch 28/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8278 - auc: 0.8977 - loss: 0.4092 - val_accuracy: 0.8563 - val_auc: 0.9234 - val_loss: 0.3686
Epoch 29/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8282 - auc: 0.8978 - loss: 0.4090 - val_accuracy: 0.8571 - val_auc: 0.9239 - val_loss: 0.3666
Epoch 30/100


5625/5625 - 8s - 1ms/step - accuracy: 0.8284 - auc: 0.8981 - loss: 0.4085 - val_accuracy: 0.8582 - val_auc: 0.9245 - val_loss: 0.3653
Epoch 31/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8283 - auc: 0.8981 - loss: 0.4087 - val_accuracy: 0.8569 - val_auc: 0.9241 - val_loss: 0.3664
Epoch 32/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8288 - auc: 0.8984 - loss: 0.4079 - val_accuracy: 0.8578 - val_auc: 0.9241 - val_loss: 0.3654
Epoch 33/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8283 - auc: 0.8982 - loss: 0.4084 - val_accuracy: 0.8582 - val_auc: 0.9245 - val_loss: 0.3656
Epoch 34/100
5625/5625 - 8s - 1ms/step - accuracy: 0.8287 - auc: 0.8984 - loss: 0.4080 - val_accuracy: 0.8587 - val_auc: 0.9246 - val_loss: 0.3654
Epoch 35/100
5625/5625 - 8s - 2ms/step - accuracy: 0.8291 - auc: 0.8987 - loss: 0.4073 - val_accuracy: 0.8591 - val_auc: 0.9249 - val_loss: 0.3657
21875/21875 ━━━━━━━━━━━━━━━━━━━━ 12s 569us/step


In [3]:
print(duration)

291.67622900009155


## Keras Tuner - 1 Attempt

## Attempt 1

In [ ]:
# Reproducibility
import os
import random
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
import keras_tuner as kt
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# Set random seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '42'
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load data
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')
sample_sub = pd.read_csv('sample_submission.csv.zip')

# Infer columns
id_col = sample_sub.columns[0]
target_col = sample_sub.columns[1]

# Prepare dataframes
df = train_df.copy()

# Encode target
e = LabelEncoder()
y = e.fit_transform(df[target_col].astype(str))

# Separate features and test set
X = df.drop(columns=[id_col, target_col], errors='ignore')
test_ids = test_df[id_col]
X_test = test_df.drop(columns=[id_col, target_col], errors='ignore')

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Feature engineering: drop columns with all missing values
X_train = X_train.dropna(axis=1, how='all')
X_val = X_val[X_train.columns]
X_test = X_test[X_train.columns]

# Identify numeric and categorical features
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = [
    col for col in X_train.select_dtypes(include=['object', 'category']).columns
    if X_train[col].nunique() <= 50
]

# Build preprocessing pipelines
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# Determine model architecture based on feature size
n_features = X_train_proc.shape[1]

# Early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# HyperModel class
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, 64)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5, 0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        x = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
        return model

# Tuner setup
bs = 32  # Example batch size
ep = 100  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False,
    project_name='bayesian_tuner'
)

# Search for the best hyperparameters
if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

# Build the best model
model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

start_time = time.time()  # Start timing

# Retrain the model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

duration = time.time() - start_time  # Calculate duration

# Log the final epoch results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Make predictions on the test set
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final_preds = (probs > 0.5).astype(int)

submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final_preds})
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 56m 49s]
val_loss: 0.28987857699394226

Best val_loss So Far: 0.28688332438468933
Total elapsed time: 10h 32m 59s
Epoch 1/100


22500/22500 - 113s - 5ms/step - accuracy: 0.8097 - auc: 0.8854 - loss: 0.4277 - val_accuracy: 0.8586 - val_auc: 0.9257 - val_loss: 0.3526
Epoch 2/100


22500/22500 - 113s - 5ms/step - accuracy: 0.8542 - auc: 0.9206 - loss: 0.3570 - val_accuracy: 0.8744 - val_auc: 0.9364 - val_loss: 0.3226
Epoch 3/100


22500/22500 - 99s - 4ms/step - accuracy: 0.8668 - auc: 0.9294 - loss: 0.3350 - val_accuracy: 0.8804 - val_auc: 0.9405 - val_loss: 0.3089
Epoch 4/100


22500/22500 - 103s - 5ms/step - accuracy: 0.8733 - auc: 0.9335 - loss: 0.3241 - val_accuracy: 0.8844 - val_auc: 0.9422 - val_loss: 0.3079
Epoch 5/100


...

22500/22500 - 91s - 4ms/step - accuracy: 0.8802 - auc: 0.9379 - loss: 0.3122 - val_accuracy: 0.8896 - val_auc: 0.9454 - val_loss: 0.2961
Epoch 8/100
22500/22500 - 96s - 4ms/step - accuracy: 0.8816 - auc: 0.9385 - loss: 0.3101 - val_accuracy: 0.8899 - val_auc: 0.9456 - val_loss: 0.2988
Epoch 9/100
22500/22500 - 93s - 4ms/step - accuracy: 0.8821 - auc: 0.9391 - loss: 0.3088 - val_accuracy: 0.8894 - val_auc: 0.9453 - val_loss: 0.2989
Epoch 10/100


22500/22500 - 105s - 5ms/step - accuracy: 0.8829 - auc: 0.9394 - loss: 0.3075 - val_accuracy: 0.8910 - val_auc: 0.9464 - val_loss: 0.2939
Epoch 11/100
22500/22500 - 109s - 5ms/step - accuracy: 0.8836 - auc: 0.9398 - loss: 0.3064 - val_accuracy: 0.8913 - val_auc: 0.9461 - val_loss: 0.2957
Epoch 12/100
22500/22500 - 87s - 4ms/step - accuracy: 0.8839 - auc: 0.9399 - loss: 0.3064 - val_accuracy: 0.8915 - val_auc: 0.9460 - val_loss: 0.2951
Epoch 13/100
22500/22500 - 95s - 4ms/step - accuracy: 0.8841 - auc: 0.9400 - loss: 0.3060 - val_accuracy: 0.8918 - val_auc: 0.9466 - val_loss: 0.2991
Epoch 14/100
22500/22500 - 101s - 4ms/step - accuracy: 0.8846 - auc: 0.9403 - loss: 0.3052 - val_accuracy: 0.8908 - val_auc: 0.9463 - val_loss: 0.2978
Epoch 15/100


22500/22500 - 108s - 5ms/step - accuracy: 0.8848 - auc: 0.9404 - loss: 0.3051 - val_accuracy: 0.8922 - val_auc: 0.9468 - val_loss: 0.2904
Epoch 16/100
22500/22500 - 99s - 4ms/step - accuracy: 0.8851 - auc: 0.9401 - loss: 0.3053 - val_accuracy: 0.8932 - val_auc: 0.9471 - val_loss: 0.2912
Epoch 17/100
22500/22500 - 98s - 4ms/step - accuracy: 0.8851 - auc: 0.9406 - loss: 0.3043 - val_accuracy: 0.8909 - val_auc: 0.9461 - val_loss: 0.2948
Epoch 18/100
22500/22500 - 111s - 5ms/step - accuracy: 0.8853 - auc: 0.9405 - loss: 0.3051 - val_accuracy: 0.8927 - val_auc: 0.9470 - val_loss: 0.2920
Epoch 19/100
22500/22500 - 103s - 5ms/step - accuracy: 0.8854 - auc: 0.9405 - loss: 0.3043 - val_accuracy: 0.8933 - val_auc: 0.9472 - val_loss: 0.2925
Epoch 20/100
22500/22500 - 102s - 5ms/step - accuracy: 0.8857 - auc: 0.9408 - loss: 0.3039 - val_accuracy: 0.8915 - val_auc: 0.9472 - val_loss: 0.2942
Epoch 21/100
22500/22500 - 101s - 4ms/step - accuracy: 0.8859 - auc: 0.9408 - loss: 0.3034 - val_accuracy: 0.

22500/22500 - 104s - 5ms/step - accuracy: 0.8866 - auc: 0.9411 - loss: 0.3029 - val_accuracy: 0.8944 - val_auc: 0.9481 - val_loss: 0.2899
Epoch 25/100
22500/22500 - 99s - 4ms/step - accuracy: 0.8865 - auc: 0.9412 - loss: 0.3026 - val_accuracy: 0.8933 - val_auc: 0.9473 - val_loss: 0.2945
Epoch 26/100
22500/22500 - 104s - 5ms/step - accuracy: 0.8858 - auc: 0.9409 - loss: 0.3035 - val_accuracy: 0.8944 - val_auc: 0.9477 - val_loss: 0.2915
Epoch 27/100


22500/22500 - 101s - 5ms/step - accuracy: 0.8865 - auc: 0.9411 - loss: 0.3036 - val_accuracy: 0.8941 - val_auc: 0.9477 - val_loss: 0.2881
Epoch 28/100
22500/22500 - 104s - 5ms/step - accuracy: 0.8867 - auc: 0.9413 - loss: 0.3027 - val_accuracy: 0.8945 - val_auc: 0.9478 - val_loss: 0.2903
Epoch 29/100
22500/22500 - 94s - 4ms/step - accuracy: 0.8864 - auc: 0.9413 - loss: 0.3029 - val_accuracy: 0.8934 - val_auc: 0.9477 - val_loss: 0.2923
Epoch 30/100
22500/22500 - 105s - 5ms/step - accuracy: 0.8869 - auc: 0.9413 - loss: 0.3021 - val_accuracy: 0.8941 - val_auc: 0.9485 - val_loss: 0.2925
Epoch 31/100
22500/22500 - 94s - 4ms/step - accuracy: 0.8873 - auc: 0.9413 - loss: 0.3022 - val_accuracy: 0.8946 - val_auc: 0.9486 - val_loss: 0.2865
Epoch 37/100
22500/22500 - 96s - 4ms/step - accuracy: 0.8877 - auc: 0.9416 - loss: 0.3022 - val_accuracy: 0.8961 - val_auc: 0.9487 - val_loss: 0.2884
Epoch 38/100
22500/22500 - 94s - 4ms/step - accuracy: 0.8877 - auc: 0.9414 - loss: 0.3018 - val_accuracy: 0.89

In [ ]:
print(duration)

4625.431574
